## Importing Required Libraries

In [ ]:
!pip install -r requirements.txt

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import regularizers
from keras.optimizers import Adam
from keras.utils import Sequence
from tensorflow import keras
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization,AveragePooling2D,Input
from keras.models import Sequential
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import numpy as np 
import cv2 
import os
import shutil

## Creating classes without the Melanose leaves

In [ ]:
classes=['Black spot', 'canker', 'greening', 'healthy'] #Creating a list of classes excluding Melanose since it should be ignored

## Creating Directories to Insert the Augmented and Testing Data

In [ ]:
def createFolder(dirName): #Creating files to store augmented and test data
    if not os.path.exists(os.path.join(dirName)): #As mentioned in the README, create the "Dateset" file and store the "Citrus" file in it
        os.mkdir(dirName)
    dirName1 = dirName+'/' 
    for folder in classes:
        dirName1=dirName1+folder
        if not os.path.exists(dirName1):
            os.mkdir(dirName1)
        dirName1 = dirName+'/'

createFolder('Dataset/TestingData')
createFolder('Dataset/AugmentedData')

## Augmenting the Data

In [ ]:
augmentor_datagen = ImageDataGenerator( 
    rotation_range=45,      #Rotating image between 0-45 degree angle
    width_shift_range=0.1,  #Flips image horizontally
    height_shift_range=0.1, #Flipms image vertically
    zoom_range=0.15,        #Zooms into image
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest",   
)

In [ ]:
num_Aug=[7,7,6,19]
def Augment(path,class_Ind):
    
    for img_name in os.listdir(path): #Loop through images in citrus/leaves folder
        img = np.expand_dims(cv2.imread(os.path.join(path,img_name)),axis=0) #Takes image from the directory
        i=num_Aug[class_Ind]
        for batch in augmentor_datagen.flow( 
            img,
            batch_size=1,
            save_to_dir=os.path.join("Dataset/AugmentedData/"+classes[class_Ind]), #Adds augmented data into Dataset folder
            save_prefix=classes[class_Ind],
            save_format='jpeg'): #Format of image

            i-=1
            if i==0:  #we generat image for each picture in batch
                break

for folder in classes:
    path = os.path.join('Dataset/Citrus/Leaves/',folder)
    class_Ind= classes.index(folder)
    Augment(path,class_Ind)

## Moving Random data from Augmented data to Testing data location

In [ ]:
def moveImages(to_path,class_Ind):
    from_dir="Dataset/AugmentedData/"+classes[Ind]
    files = os.listdir(from_dir)
    # Select 0.2 of the files randomly 
    random_files = np.random.choice(files, int(len(files)*.2))

    # moving 10 files from each class to testing data file
    i=0
    for x in random_files:
        shutil.move(from_dir+'/'+x, to_path) # move file to anthore dir
        i+=1
        if i==20:
            break

for folder in classes:
    path = os.path.join('Dataset/TestingData/',folder)#Adding images to dateset/testingdata folder
    Ind= classes.index(folder)
    moveImages(path,Ind)

In [ ]:
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    #80%-20% split
    validation_split=0.2, 
)
#training data
train_generator = training_datagen.flow_from_directory( 
    directory='Dataset/AugmentedData/', 
    target_size=(256,256), #imgae size 256x256
    shuffle=True, 
    batch_size=64,
    subset='training',
)
#validtaion data
validation_generator = training_datagen.flow_from_directory(
    directory='Dataset/AugmentedData/',
    target_size=(256,256),
    shuffle=True, 
    batch_size=64,
    subset='validation',
)



## ResNet Model

In [ ]:
model = Sequential([ #Resnet model with 5 convolutional layers and 5 MaxPool layers
    #Input Layer 
    Conv2D(32, (3,3), padding='same',activation='relu',input_shape=(256, 256, 3),),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3),activation='relu', padding='same', ),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(128, (3,3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),

   
    Conv2D(256, (3,3), activation='relu', padding='same', ),
    MaxPooling2D(pool_size=(2,2)),

    
    Conv2D(512, (3,3), activation='relu', padding='same', ),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dropout(0.25),
    
    #Connection Layer
    Dense(512,activation='relu', activity_regularizer=regularizers.l2(1e-5)),
    Dense(4,activation='softmax'),
    
])
model.summary()

In [ ]:
plot_model(model)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
con = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
ResNet = model.fit(train_generator, epochs=50, validation_data = validation_generator, callbacks=[con])

## Plotting the Model Accuracy and Loss

In [ ]:
#Plotting the accuracy/epoch of Resnet Model
plt.plot(ResNet.history['accuracy'])
plt.plot(ResNet.history['val_accuracy'])
plt.title('ResNet Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.rcParams["figure.figsize"] = (20,3)
plt.show()
# Plotting loss/epoch of resnet model
plt.plot(ResNet.history['loss'])
plt.plot(ResNet.history['val_loss'])
plt.title('ResNet Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### LeNet Model

In [ ]:
model2 = keras.Sequential([
    
    Conv2D(32, (3,3), padding='same',activation='relu',input_shape=(256, 256, 3),),
    
    Conv2D(filters=6, kernel_size=(5, 5),activation='sigmoid'),
    AveragePooling2D(pool_size=(2, 2), strides=(2, 2)),

    Conv2D(filters=16, kernel_size=(5, 5), activation='sigmoid'),
    AveragePooling2D(pool_size=(2, 2), strides=(2, 2)),

    Flatten(),

    Dense(units=120, activation='sigmoid'),
    Dense(units=84, activation='sigmoid'),
    Dense(units=10, activation='softmax')
])
model2.summary()

In [ ]:
plot_model(model2)

In [ ]:
model2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
con = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
LeNet = model.fit(train_generator, epochs=50, validation_data = validation_generator, callbacks=[con])

## Plotting the Model Accuracy and Loss

In [ ]:
# summarize history for accuracy
plt.plot(LeNet.history['accuracy'])
plt.plot(LeNet.history['val_accuracy'])
plt.title('ResNet Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.rcParams["figure.figsize"] = (20,3)
plt.show()
# summarize history for loss
plt.plot(LeNet.history['loss'])
plt.plot(LeNet.history['val_loss'])
plt.title('ResNet Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### AlexNet Model

In [ ]:
model3 = keras.Sequential([

    Input(shape=(256, 256, 3)),

    Conv2D(filters=96, kernel_size=(11, 11),
           strides=(4, 4), activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

    Conv2D(filters=256, kernel_size=(5, 5), 
           strides=(1, 1), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

    Conv2D(filters=384, kernel_size=(3, 3), 
           strides=(1, 1), padding='same', activation='relu'),
    Conv2D(filters=384, kernel_size=(3, 3), 
           strides=(1, 1), padding='same', activation='relu'),
    Conv2D(filters=256, kernel_size=(3, 3), 
           strides=(1, 1), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

    Flatten(),

    Dense(units=4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(1000, activation='softmax')
])
model3.summary()

In [ ]:
plot_model(model3)

In [ ]:
model3.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
con = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
AlexNet = model.fit(train_generator, epochs=50, validation_data = validation_generator, callbacks=[con])

## Plotting the Model Accuracy and Loss

In [ ]:
# summarize history for accuracy
plt.plot(AlexNet.history['accuracy'])
plt.plot(AlexNet.history['val_accuracy'])
plt.title('ResNet Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.rcParams["figure.figsize"] = (20,3)
plt.show()
# summarize history for loss
plt.plot(AlexNet.history['loss'])
plt.plot(AlexNet.history['val_loss'])
plt.title('ResNet Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## VGG16 Model

In [ ]:
model4 = keras.Sequential([  
    Input(shape=(256,256,3)),
    
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(1000, activation='softmax')
])
model4.summary()

In [ ]:
plot_model(model4)

In [ ]:
model4.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
con = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
VGG16 = model.fit(train_generator, epochs=50, validation_data = validation_generator, callbacks=[con])

## Plotting the Model Accuracy and Loss

In [ ]:
# summarize history for accuracy
plt.plot(VGG16.history['accuracy'])
plt.plot(VGG16.history['val_accuracy'])
plt.title('ResNet Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.rcParams["figure.figsize"] = (20,3)
plt.show()
# summarize history for loss
plt.plot(VGG16.history['loss'])
plt.plot(VGG16.history['val_loss'])
plt.title('ResNet Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
image_gen = ImageDataGenerator(rescale = 1./255) #Rescalling Image
image_pred = image_gen.flow_from_directory("Dataset/TestingData/" ,target_size=(256,256),batch_size=5,shuffle=False,class_mode='categorical',)
image_pred.reset()


### Confusion Matrix

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix

def plot_confusion_matrix(Cm, classes,title='Confusion matrix',Cmap=plt.cm.Blues):
    plt.imshow(Cm, interpolation='nearest', Cmap=Cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)


    thresh = Cm.max() / 2.
    for i, j in itertools.product(range(Cm.shape[0]), range(Cm.shape[1])):
        plt.text(j, i, Cm[i, j],
                 horizontalalignment="center",
                 color="white" if Cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
Y_predictions = model.predict_generator(image_pred)
Y_predicted_class_indices= np.argmax(Y_predictions,axis=1)
cm=confusion_matrix(image_pred.classes, Y_predicted_class_indices)
plot_confusion_matrix(cm, classes=classes,title='Leave Disease Classification')
plt.show()